In [1]:
import os
from datetime import datetime

import folium
import serial
import pandas as pd
import numpy as np

In [2]:
def parse_packet(packet: bytes) -> tuple:
    print(packet)
    try:
        packet = packet.decode()
        start = packet.index("'") +1
        end = packet.index("'",start)
        data = packet[start:end]
        lat, lon, altitude = data.split(",")
    except (ValueError, AttributeError):
        return np.nan, np.nan, np.nan
    try:
        int(altitude)
    except ValueError:
        altitude = np.nan
    try:
        return float(lat), float(lon), altitude
    except ValueError:
        lat, lon = np.nan
        return lat, lon, altitude

def read_existing_track(filename: str) -> tuple:
    existing = pd.read_csv(filename, header=None)
    times = list(existing.iloc[:, 0])
    lats = list(existing.iloc[:, 1])
    lons = list(existing.iloc[:, 2])
    return lats, lons, times

def save_track(lat, lon, alt, timestamp, filename):
    with open(filename, "a") as f:
        f.write(f"{timestamp},{lat},{lon},{alt}\n")

In [3]:
existing_track = "track.csv"
m = folium.Map(location=[37.2, -120.5], zoom_start=10)
if os.path.isfile(existing_track):
    lats, lons, times = read_existing_track(existing_track)
    folium.PolyLine([(i, j) for i,j in zip(lats, lons)]).add_to(m)
    for i,j,k in zip(lats, lons, times):
        folium.Marker([i, j], tooltip=k).add_to(m)
else:
    lats = []
    lons = []
    times = []

i = 1
with serial.Serial("/dev/ttyACM0", 9600) as ser:
    while True:
        packet = ser.readline()
        received_time = datetime.now()
        lat, lon, alt = parse_packet(packet)
        timestamp = received_time.isoformat()
        if not (np.isnan(lat) or np.isnan(lon)):
            tooltip = f"{timestamp}, {alt} meters"
            lats.append(lat)
            lons.append(lon)
            times.append(received_time)
            folium.Marker([lat, lon], tooltip=tooltip).add_to(m)
            print(i, timestamp, lat, lon, alt)
            if len(lats) > 1:
                folium.PolyLine([(lats[-2], lons[-2]), (lat, lon)]).add_to(m)
            m.save("index.html")
            save_track(lat, lon, alt, timestamp, existing_track)
            i += 1

b"Received packet '\x12q\x0e\xfc!\xcd\xb6rt\xff\xb7\rd\xe2\xf6\x8e{\xb4\xb0)\xcd' with RSSI -114\r\n"
b"Received packet '36.5720,-121.5173,33' with RSSI -71\r\n"
1 2021-06-12T12:52:35.466051 36.572 -121.5173 33
b"Received packet '36.5720,-121.5173,33' with RSSI -77\r\n"
2 2021-06-12T12:52:50.448588 36.572 -121.5173 33
b"Received packet '36.5720,-121.5173,33' with RSSI -84\r\n"
3 2021-06-12T12:53:05.539454 36.572 -121.5173 33
b"Received packet '36.5720,-121.5173,33' with RSSI -70\r\n"
4 2021-06-12T12:53:20.617474 36.572 -121.5173 33
b"Received packet '36.5720,-121.5173,33' with RSSI -69\r\n"
5 2021-06-12T12:53:35.672412 36.572 -121.5173 33
b"Received packet '36.5720,-121.5173,33' with RSSI -72\r\n"
6 2021-06-12T12:53:50.727028 36.572 -121.5173 33
b"Received packet '36.5720,-121.5173,33' with RSSI -71\r\n"
7 2021-06-12T12:54:05.707163 36.572 -121.5173 33
b"Received packet '36.5720,-121.5173,33' with RSSI -67\r\n"
8 2021-06-12T12:54:20.792930 36.572 -121.5173 33
b"Received packet '36.5720

KeyboardInterrupt: 